##### 참조 
- https://hleecaster.com/ml-logistic-regression-concept/

 ### 로지스틱 회귀(Logistic Regression)
- 로지스틱 회귀는 선형 회귀 방식을 분류에 적용한 알고리즘이다.
- 로지스틱 회귀 역시 가중치(weight) 변수가 선형이므로 선형 회귀 계열이다. 
- 로지스틱 회귀가 선형 회귀와 다른 점은 학습을 통해 선형 함수의 회귀 최적선을 찾는 것이 아니라 시그모이드(Sigmoid) 함수 최적선을 찾고 이 시그모이드 함수의 반환 값을 확률로 간주해 확률에 따라 분류를 결정한다는 것이다.
   - 즉, 로지스틱 회귀는 선형 회귀 방식을 기반으로 하되 시그모이드 함수를 이용해 분류를 수행하는 회귀이다. 

<img src='./dataset/sigmoid.png'>

- 시그모이드 함수는 x 값이 아무리 커지거나 작아져도 y 값은 항상 0과 1 사이 값을 반환한다.
- x 값이 커지면 1에 근사하며 x 값이 작아지면 0에 근사한다 그리고 x가 0일 때는 0.5이다. 
- 사이킷런에서 로지스틱 회귀는 LogisticRegression() 클래스를 통해 구현되는데, 주요 하이퍼 파라미터로는 penalty와 C가 있다.
- penalty는 규제의 유형을 설정하며 'l1'은 L1 규제를, 'l2'는 L2 규제를 적용하게 된다. 디폴트는 'l2'이다. C는 규제 강도를 조절하는 alpha 값의 역수이다. C 값이 작을수록 규제 강도가 크다.
    - 규제 강도가 크다는 것은 W(회귀 계수)가 작아짐을 의미한다.</BR></BR>

- 로지스틱 회귀는 가볍고 빠르며 이진 분류 예측 성능이 뛰어나다. 이 때문에 이진 분류의 기본 모델로 로지스틱 회귀를 사용하는 경우가 많다. 
- 또한 로지스틱 회귀는 희소한 데이터 세트 분류에도 뛰어난 성능을 보여서 텍스트 분류에서도 자주 사용된다.

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')
 
# 데이터 로딩
cancer = load_breast_cancer()

# 표준화 -> 특성들의 평균을 0, 분산을 1로 스케일링 -> 즉, 특성들을 정규분포로 만드는 것이다
scaler = StandardScaler()
# 학습데이터 세트에서 변환을 위한 기반 설정 -> (예를 들어 학습 데이터 세트의 최대값/최소값등)을 먼저 fit()을 통해서 설정
# 이를 기반으로 학습 데이터의 transform()을 수행하되 학습 데이터에서 스케일 되어진 데이터를 그대로 테스트 데이터에도 적용하기 위함이다 
# 즉 학습 데이터 세트로 fit()된 Scaler를 이용하여 테스트 데이터를 변환할 경우에는 테스트 데이터에서 다시 fit()하지 않고
# 반드시 그대로 전 Scaler를 이용하여 transform()을 수행해야 합니다.
# 학습 데이터에서 Scale된 데이터를 기반으로 Classifier가 학습이 되었기 때문에 
# 이렇게 학습된 Classifier가 예측을 할 때에도 학습 데이터의 Scale 기준으로 테스트 데이터를 변환 한 뒤 predict해야 합니다.
data_scaled = scaler.fit_transform(cancer.data)

# 테스트 분리
X_train, X_test, y_train, y_test = train_test_split(data_scaled,cancer.target,test_size=0.3,random_state=0)

# 로지스틱 회귀를 이용해 학습 및 예측 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)
lr_preds = lr_clf.predict(X_test)
lr_preds_proba = lr_clf.predict_proba(X_test)[:,1] # X_test로 여러건이 입력되므로 적용

# 정확도와 roc_auc 측정
accuracy = accuracy_score(y_test,lr_preds)
roc_auc = roc_auc_score(y_test,lr_preds_proba)
print(f'accuracy: {accuracy:.4f}')
print(f'roc_auc: {roc_auc:.4f}')

accuracy: 0.9766
roc_auc: 0.9947


In [9]:
from sklearn.model_selection import GridSearchCV

# 사이킷런 LogisticRegression 클래스의 주요 하이퍼 파라미터로 penalty 와 C가 있다
# penalty는 규제의 유형을 설정 C는 규제강도를 조절하는 alpha값의 역수

params = {'penalty':['l2','l1'],
         'C':[0.01,0.1,1,5,10]}

grid_clf = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=3)
grid_clf.fit(data_scaled,cancer.target)
print(f'최적 하이퍼 파라미터: {grid_clf.best_params_}')
print(f'최적 평균 정확도: {grid_clf.best_score_}')

pred = grid_clf.best_estimator_.predict(X_test)
accuracy_test = accuracy_score(y_test,pred)
print(f'테스트 데이터 정확도: {accuracy_test:.4f}')
# 로지스틱 회귀는 가볍고 빠르지만, 이진 분류 예측 성능도 뛰어나다
# 이렇기 때문에 로지스틱 회귀를 이진 분류의 기본 모델로 사용하는 경우가 많다. -> text분류 등

최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}
최적 평균 정확도: 0.975392184164114
테스트 데이터 정확도: 0.9883
